In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

import time

from math import sqrt
np.random.seed(1234)

In [ ]:
ball = pd.read_csv('Half Ball Prediction Data.csv')
convex = pd.read_csv('Convex Prediction Data.csv')
freeform = pd.read_csv('Freeform 2 Prediction Data.csv')

test_data = pd.read_csv("Whole Test Grid.csv")

test_total = []
for i in range(1,51):
    nameX = "Scan" + str(i) + "X"
    nameY = "Scan" + str(i) + "Y"
    nameZ = "Scan" + str(i) + "Z"
    nameCurv = "Scan" + str(i) + "curvature"
    nameAngle = "Scan" + str(i) + "angle"
    scan_i = test_data[[nameX, nameY, nameZ,nameCurv,nameAngle]]
    scan_i.columns = ['X', 'Y', 'Z','curvature','angle']
    test_total.append(scan_i)

test_d = test_total[2]
    
true_variance = pd.read_csv('True Variance.csv')

In [ ]:
def hyperbolic(x):
    return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))

def generate_neurons_uniform(X,num_neurons):
    k = X.shape[1]
    variances = np.std(X,axis=0)
    weights = []
    for i in range(k):
        λ = variances[i]/np.sum(variances)
        a = 2.5 * λ / np.max(np.abs(X[:,i]))
        weight = np.random.uniform(low=-a,high=a,size=num_neurons)
        weights.append(weight)
    weights = np.array(weights)
    intercept = np.random.uniform(low=-1,high=1,size=num_neurons)
    
    return weights, intercept

def cal_neuron_val(X, weights, intercept):
    neuron_val = np.matmul(X,weights) + intercept
    neuron_val = hyperbolic(neuron_val)
    return neuron_val

In [ ]:
# BELM
def mbelm(data,coordinate,num_neurons=50):
    weights, intercept = generate_neurons_uniform(data[['Mean Curvature','Mean Angle']].values,num_neurons=num_neurons)
    neuron_val = cal_neuron_val(data[['Mean Curvature','Mean Angle']].values,weights,intercept)
    clf = BayesianRidge()
    name = 'Pointwise Variance ' + coordinate
    clf.fit(neuron_val, data[name].values)
    return {'weight':weights,'intercept':intercept,'model':clf}

def mbelm_pred(train,model):
    neuron_pred = cal_neuron_val(train[['curvature','angle']].values, model['weight'],model['intercept'])
    return model['model'].predict(neuron_pred)

In [ ]:
# E-BELM
def mbelm_ensemble(data,coordinate,num_neurons=3, ensemble_num = 50):
    ensemble_weights = []
    ensemble_intercept = []
    ensemble_model = []
    
    for i in range(ensemble_num):
    
        weights, intercept = generate_neurons_uniform(data[['Mean Curvature','Mean Angle']].values,
                                                      num_neurons=num_neurons)
        neuron_val = cal_neuron_val(data[['Mean Curvature','Mean Angle']].values,weights,intercept)

        # Fit the output layer
        clf = BayesianRidge()
        name = 'Pointwise Variance ' + coordinate
        clf.fit(neuron_val, data[name].values)
        
        ensemble_weights.append(weights)
        ensemble_intercept.append(intercept)
        ensemble_model.append(clf)
    
    return {'weight':ensemble_weights,'intercept':ensemble_intercept, 
            'model':ensemble_model, "num": ensemble_num}

def mbelm_pred_ensemble(train,model):
    prediction = 0
    for i in range(model['num']):
        neuron_pred = cal_neuron_val(train[['curvature','angle']].values, 
                                     model['weight'][i],
                                     model['intercept'][i])
        prediction += model['model'][i].predict(neuron_pred)
        
    prediction = prediction/model['num']
    
    return prediction

In [ ]:
# Res-BELM
def residual_mbelm(data,coordinate,num_neurons=40, num_residual_nets=5):
    nets_weights = []
    nets_intercept = []
    nets_model = []
    
    name = 'Pointwise Variance ' + coordinate
    
    residual = data[name].values
    
    for i in range(num_residual_nets):
    
        weights, intercept = generate_neurons_uniform(data[['Mean Curvature','Mean Angle']].values,
                                                      num_neurons=num_neurons)
        neuron_val = cal_neuron_val(data[['Mean Curvature','Mean Angle']].values,weights,intercept)

        # Fit the output layer
        clf = BayesianRidge()
        
        clf.fit(neuron_val, residual)
        
        residual = residual - clf.predict(neuron_val)
        
        nets_weights.append(weights)
        nets_intercept.append(intercept)
        nets_model.append(clf)
    
    return {'weight':nets_weights,'intercept': nets_intercept, 
            'model':nets_model, "num": num_residual_nets}

def residual_mbelm_pred(train,model):
    prediction = 0
    for i in range(model['num']):
        neuron_pred = cal_neuron_val(train[['curvature','angle']].values, 
                                     model['weight'][i],
                                     model['intercept'][i])
        
        prediction += model['model'][i].predict(neuron_pred)
        
    prediction = prediction
    
    return prediction

In [ ]:
# Training sets
ball = pd.read_csv('Half Ball Prediction Data.csv')
convex = pd.read_csv('Convex Prediction Data.csv')
freeform = pd.read_csv('Freeform 2 Prediction Data.csv')

ball['Pointwise Variance X'] = np.log(ball['Pointwise Variance X'])
ball['Pointwise Variance Y'] = np.log(ball['Pointwise Variance Y'])
ball['Pointwise Variance Z'] = np.log(ball['Pointwise Variance Z'])

convex['Pointwise Variance X'] = np.log(convex['Pointwise Variance X'])
convex['Pointwise Variance Y'] = np.log(convex['Pointwise Variance Y'])
convex['Pointwise Variance Z'] = np.log(convex['Pointwise Variance Z'])

freeform['Pointwise Variance X'] = np.log(freeform['Pointwise Variance X'])
freeform['Pointwise Variance Y'] = np.log(freeform['Pointwise Variance Y'])
freeform['Pointwise Variance Z'] = np.log(freeform['Pointwise Variance Z'])

In [ ]:
# ball_downsample = ball.sample(n=10000)
# ball_downsample = convex.sample(n=10000)
ball_downsample = freeform.sample(n=10000)

In [ ]:
ball_feature = ball_downsample[['Mean Curvature','Mean Angle']].values
ball_target_x = ball_downsample['Pointwise Variance X'].values
ball_target_y = ball_downsample['Pointwise Variance Y'].values
ball_target_z = ball_downsample['Pointwise Variance Z'].values

In [ ]:
test_feature = test_d[['curvature', 'angle']].values

In [ ]:
# Neural Networks

## x-coordinate

In [ ]:
%timeit regr = MLPRegressor(random_state=1234, max_iter=1000).fit(ball_feature, ball_target_x)

In [ ]:
%timeit regr.predict(test_feature)

In [ ]:
print(sqrt(mean_squared_error(true_variance['X Variance'], np.exp(regr.predict(test_feature)))))

In [ ]:
## y-coordinate

In [ ]:
%timeit regr = MLPRegressor(random_state=1234, max_iter=1000).fit(ball_feature, ball_target_y)

In [ ]:
%timeit regr.predict(test_feature)

In [ ]:
print(sqrt(mean_squared_error(true_variance['Y Variance'], np.exp(regr.predict(test_feature)))))

In [ ]:
## z-coordinate

In [ ]:
%timeit regr = MLPRegressor(random_state=1234, max_iter=1000).fit(ball_feature, ball_target_z)

In [ ]:
regr = MLPRegressor(random_state=1234, max_iter=1000).fit(ball_feature, ball_target_z)

In [ ]:
%timeit regr.predict(test_feature)

In [ ]:
print(sqrt(mean_squared_error(true_variance['Z Variance'], np.exp(regr.predict(test_feature)))))

In [ ]:
# Gaussian Process

## x-coordinate

In [ ]:
%timeit gpr = GaussianProcessRegressor(kernel=RBF, random_state=0).fit(ball_feature, ball_target_x)

In [ ]:
gpr = GaussianProcessRegressor(random_state=1234).fit(ball_feature, ball_target_x)

In [ ]:
%timeit gpr.predict(test_feature)

In [ ]:
print(sqrt(mean_squared_error(true_variance['X Variance'], np.exp(gpr.predict(test_feature)))))

In [ ]:
## y-coordinate

In [ ]:
%timeit gpr = GaussianProcessRegressor(random_state=1234).fit(ball_feature, ball_target_y)

In [ ]:
gpr = GaussianProcessRegressor(random_state=1234).fit(ball_feature, ball_target_y)

In [ ]:
%timeit gpr.predict(test_feature)

In [ ]:
print(sqrt(mean_squared_error(true_variance['Y Variance'], np.exp(gpr.predict(test_feature)))))

In [ ]:
## z-coordinate

In [ ]:
%timeit gpr = GaussianProcessRegressor(random_state=1234).fit(ball_feature, ball_target_z)

In [ ]:
gpr = GaussianProcessRegressor(random_state=1234).fit(ball_feature, ball_target_y)

In [ ]:
%timeit gpr.predict(test_feature)

In [ ]:
print(sqrt(mean_squared_error(true_variance['Z Variance'], np.exp(gpr.predict(test_feature)))))

In [ ]:
# Random Forests

## x-coordinate

In [ ]:
%timeit clf = RandomForestRegressor(random_state=1234).fit(ball_feature, ball_target_x)

In [ ]:
clf = RandomForestRegressor(random_state=1234).fit(ball_feature, ball_target_x)

In [ ]:
%timeit clf.predict(test_feature)

In [ ]:
print(sqrt(mean_squared_error(true_variance['X Variance'], np.exp(clf.predict(test_feature)))))

In [ ]:
## y-coordiante

In [ ]:
%timeit clf = RandomForestRegressor(random_state=1234).fit(ball_feature, ball_target_y)

In [ ]:
clf = RandomForestRegressor(random_state=1234).fit(ball_feature, ball_target_y)

In [ ]:
%timeit clf.predict(test_feature)

In [ ]:
print(sqrt(mean_squared_error(true_variance['Y Variance'], np.exp(clf.predict(test_feature)))))

In [ ]:
## z-coordiante

In [ ]:
%timeit clf = RandomForestRegressor(random_state=1234).fit(ball_feature, ball_target_z)

In [ ]:
clf = RandomForestRegressor(random_state=1234).fit(ball_feature, ball_target_z)

In [ ]:
%timeit clf.predict(test_feature)

In [ ]:
print(sqrt(mean_squared_error(true_variance['Z Variance'], np.exp(clf.predict(test_feature)))))

In [ ]:
# Support Vector Machine

## x-coordinate

In [ ]:
%timeit regr = SVR().fit(ball_feature, ball_target_x)

In [ ]:
regr = SVR().fit(ball_feature, ball_target_x)

In [ ]:
%timeit regr.predict(test_feature)

In [ ]:
print(sqrt(mean_squared_error(true_variance['X Variance'], np.exp(regr.predict(test_feature)))))

In [ ]:
## y-coordiante

In [ ]:
%timeit regr = SVR().fit(ball_feature, ball_target_y)

In [ ]:
regr = SVR().fit(ball_feature, ball_target_y)

In [ ]:
%timeit regr.predict(test_feature)

In [ ]:
print(sqrt(mean_squared_error(true_variance['Y Variance'], np.exp(regr.predict(test_feature)))))

In [ ]:
## z-coordinate

In [ ]:
%timeit regr = SVR().fit(ball_feature, ball_target_z)

In [ ]:
regr = SVR().fit(ball_feature, ball_target_z)

In [ ]:
%timeit regr.predict(test_feature)

In [ ]:
print(sqrt(mean_squared_error(true_variance['Z Variance'], np.exp(regr.predict(test_feature)))))

In [ ]:
# BELM

## x-coordiante

In [ ]:
%timeit belm_x = mbelm(ball_downsample,'X')

In [ ]:
belm_x = mbelm(ball_downsample,'X')

In [ ]:
%timeit mbelm_pred(test_d, belm_x)

In [ ]:
print(sqrt(mean_squared_error(true_variance['X Variance'], np.exp(mbelm_pred(test_d,belm_x)))))

In [ ]:
## y-coordiante

In [ ]:
%timeit belm_y = mbelm(ball_downsample,'Y')

In [ ]:
belm_y = mbelm(ball_downsample,'Y')

In [ ]:
%timeit mbelm_pred(test_d, belm_y)

In [ ]:
print(sqrt(mean_squared_error(true_variance['Y Variance'], np.exp(mbelm_pred(test_d,belm_y)))))

In [ ]:
## z-coordinate

In [ ]:
%timeit belm_z = mbelm(ball_downsample,'Z')

In [ ]:
belm_z = mbelm(ball_downsample,'Z')

In [ ]:
%timeit mbelm_pred(test_d, belm_z)

In [ ]:
print(sqrt(mean_squared_error(true_variance['Z Variance'], np.exp(mbelm_pred(test_d,belm_z)))))

In [ ]:
# E-BELM

## x-coordinate

In [ ]:
%timeit e_belm_x = mbelm_ensemble(ball_downsample,'X')

In [ ]:
e_belm_x = mbelm_ensemble(ball_downsample,'X')

In [ ]:
%timeit mbelm_pred_ensemble(test_d, e_belm_x)

In [ ]:
print(sqrt(mean_squared_error(true_variance['X Variance'], np.exp(mbelm_pred_ensemble(test_d, e_belm_x)))))

In [ ]:
## y-coordiante

In [ ]:
%timeit e_belm_y = mbelm_ensemble(ball_downsample,'Y')

In [ ]:
e_belm_y = mbelm_ensemble(ball_downsample,'Y')

In [ ]:
%timeit mbelm_pred_ensemble(test_d, e_belm_y)

In [ ]:
print(sqrt(mean_squared_error(true_variance['Y Variance'], np.exp(mbelm_pred_ensemble(test_d, e_belm_y)))))

In [ ]:
## z-coordinate

In [ ]:
%timeit e_belm_z = mbelm_ensemble(ball_downsample,'Z')

In [ ]:
e_belm_z = mbelm_ensemble(ball_downsample,'Z')

In [ ]:
%timeit mbelm_pred_ensemble(test_d, e_belm_z)

In [ ]:
print(sqrt(mean_squared_error(true_variance['Z Variance'], np.exp(mbelm_pred_ensemble(test_d, e_belm_z)))))

In [ ]:
# Res-BELM

## x-coordinate

In [ ]:
%timeit res_belm_x = residual_mbelm(ball_downsample,'X')

In [ ]:
res_belm_x = residual_mbelm(ball_downsample,'X')

In [ ]:
%timeit residual_mbelm_pred(test_d, res_belm_x)

In [ ]:
print(sqrt(mean_squared_error(true_variance['X Variance'], np.exp(residual_mbelm_pred(test_d, res_belm_x)))))

In [ ]:
## y-coordinate

In [ ]:
%timeit res_belm_y = residual_mbelm(ball_downsample,'Y')

In [ ]:
res_belm_y = residual_mbelm(ball_downsample,'Y')

In [ ]:
%timeit residual_mbelm_pred(test_d, res_belm_y)

In [ ]:
print(sqrt(mean_squared_error(true_variance['Y Variance'], np.exp(residual_mbelm_pred(test_d, res_belm_y)))))

In [ ]:
## z-coordinate

In [ ]:
%timeit res_belm_z = residual_mbelm(ball_downsample,'Z')

In [ ]:
res_belm_z = residual_mbelm(ball_downsample,'Z')

In [ ]:
%timeit residual_mbelm_pred(test_d, res_belm_z)

In [ ]:
print(sqrt(mean_squared_error(true_variance['Z Variance'], np.exp(residual_mbelm_pred(test_d, res_belm_z)))))